In [40]:
#from google.colab import drive
#drive.mount('/content/drive')

In [41]:
#%cd /content/drive/MyDrive/TemporAi

In [42]:
#!pip install darts --quiet

In [43]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from darts.models.forecasting.lgbm import LightGBMModel

In [44]:
#!pip install darts --quiet

In [45]:
from darts import TimeSeries

In [46]:
!ls

Darts_hospital_data.ipynb	  README.md
datasets			  StatsForecast_all_datasets.ipynb
LICENSE				  TemporaAIDartsModels-hospital_data.ipynb
lightGBM_air_quality.ipynb	  TemporaAIDartsModels.ipynb
lightGBM_temp_anamoly_data.ipynb  TemporAI_Darts_lightGBM_home_dataset.ipynb
MTRNotTracked			  ts_utils.py
__pycache__			  utility.py


In [47]:
n_days_to_consider = 349
df = pd.read_csv('datasets/HomeC.csv', nrows=60*24*n_days_to_consider)

/tmp/ipykernel_7137/1506045390.py:2: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('datasets/HomeC.csv', nrows=60*24*n_days_to_consider)


In [48]:
df

,time,use [kW],gen [kW],House overall [kW],Dishwasher [kW],Furnace 1 [kW],Furnace 2 [kW],Home office [kW],Fridge [kW],Wine cellar [kW],...,visibility,summary,apparentTemperature,pressure,windSpeed,cloudCover,windBearing,precipIntensity,dewPoint,precipProbability
0,1451624400,0.932833,0.003483,0.932833,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,...,10.00,Clear,29.26,1016.91,9.18,cloudCover,282,0.0,24.40,0.0
1,1451624401,0.934333,0.003467,0.934333,0.000000,0.020717,0.063817,0.444067,0.124000,0.006983,...,10.00,Clear,29.26,1016.91,9.18,cloudCover,282,0.0,24.40,0.0
2,1451624402,0.931817,0.003467,0.931817,0.000017,0.020700,0.062317,0.446067,0.123533,0.006983,...,10.00,Clear,29.26,1016.91,9.18,cloudCover,282,0.0,24.40,0.0
3,1451624403,1.022050,0.003483,1.022050,0.000017,0.106900,0.068517,0.446583,0.123133,0.006983,...,10.00,Clear,29.26,1016.91,9.18,cloudCover,282,0.0,24.40,0.0
4,1451624404,1.139400,0.003467,1.139400,0.000133,0.236933,0.063983,0.446533,0.122850,0.006850,...,10.00,Clear,29.26,1016.91,9.18,cloudCover,282,0.0,24.40,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502555,1452126955,1.181467,0.003400,1.181467,0.093800,0.086100,0.065550,0.042533,0.137600,0.015217,...,9.17,Clear,24.80,995.90,9.44,0.0,279,0.0,27.46,0.0
502556,1452126956,0.982050,0.003700,0.982050,0.119450,0.102067,0.065300,0.042483,0.137067,0.014900,...,9.17,Clear,24.80,995.90,9.44,0.0,279,0.0,27.46,0.0
502557,1452126957,1.178700,0.004033,1.178700,0.119750,0.498167,0.063400,0.042717,0.136933,0.014883,...,9.17,Clear,24.80,995.90,9.44,0.0,279,0.0,27.46,0.0
502558,1452126958,1.383217,0.004050,1.383217,0.119733,0.497683,0.064750,0.042683,0.136200,0.015417,...,9.17,Clear,24.80,995.90,9.44,0.0,279,0.0,27.46,0.0


In [49]:
len(df), n_days_to_consider*24*60

(502560, 502560)

In [50]:
df.head(5)

,time,use [kW],gen [kW],House overall [kW],Dishwasher [kW],Furnace 1 [kW],Furnace 2 [kW],Home office [kW],Fridge [kW],Wine cellar [kW],...,visibility,summary,apparentTemperature,pressure,windSpeed,cloudCover,windBearing,precipIntensity,dewPoint,precipProbability
0,1451624400,0.932833,0.003483,0.932833,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,...,10.0,Clear,29.26,1016.91,9.18,cloudCover,282,0.0,24.4,0.0
1,1451624401,0.934333,0.003467,0.934333,0.000000,0.020717,0.063817,0.444067,0.124000,0.006983,...,10.0,Clear,29.26,1016.91,9.18,cloudCover,282,0.0,24.4,0.0
2,1451624402,0.931817,0.003467,0.931817,0.000017,0.020700,0.062317,0.446067,0.123533,0.006983,...,10.0,Clear,29.26,1016.91,9.18,cloudCover,282,0.0,24.4,0.0
3,1451624403,1.022050,0.003483,1.022050,0.000017,0.106900,0.068517,0.446583,0.123133,0.006983,...,10.0,Clear,29.26,1016.91,9.18,cloudCover,282,0.0,24.4,0.0
4,1451624404,1.139400,0.003467,1.139400,0.000133,0.236933,0.063983,0.446533,0.122850,0.006850,...,10.0,Clear,29.26,1016.91,9.18,cloudCover,282,0.0,24.4,0.0


In [51]:
import datetime

base_date = datetime.date(2022, 1, 1)  # Replace with your desired base date
time_deltas = pd.to_timedelta((df['time'] - df['time'][0]) * 1, unit='minutes')

In [52]:
time_deltas

0          0 days 00:00:00
1          0 days 00:01:00
2          0 days 00:02:00
3          0 days 00:03:00
4          0 days 00:04:00
                ...       
502555   348 days 23:55:00
502556   348 days 23:56:00
502557   348 days 23:57:00
502558   348 days 23:58:00
502559   348 days 23:59:00
Name: time, Length: 502560, dtype: timedelta64[ns]

In [53]:
df['datetime_column'] = pd.to_datetime(base_date) + time_deltas


In [54]:
df['datetime_column']

0        2022-01-01 00:00:00
1        2022-01-01 00:01:00
2        2022-01-01 00:02:00
3        2022-01-01 00:03:00
4        2022-01-01 00:04:00
                 ...        
502555   2022-12-15 23:55:00
502556   2022-12-15 23:56:00
502557   2022-12-15 23:57:00
502558   2022-12-15 23:58:00
502559   2022-12-15 23:59:00
Name: datetime_column, Length: 502560, dtype: datetime64[ns]

In [55]:
df.columns

Index(['time', 'use [kW]', 'gen [kW]', 'House overall [kW]', 'Dishwasher [kW]',
       'Furnace 1 [kW]', 'Furnace 2 [kW]', 'Home office [kW]', 'Fridge [kW]',
       'Wine cellar [kW]', 'Garage door [kW]', 'Kitchen 12 [kW]',
       'Kitchen 14 [kW]', 'Kitchen 38 [kW]', 'Barn [kW]', 'Well [kW]',
       'Microwave [kW]', 'Living room [kW]', 'Solar [kW]', 'temperature',
       'icon', 'humidity', 'visibility', 'summary', 'apparentTemperature',
       'pressure', 'windSpeed', 'cloudCover', 'windBearing', 'precipIntensity',
       'dewPoint', 'precipProbability', 'datetime_column'],
      dtype='object')

In [56]:
df_resampled = df.resample('h', on='datetime_column').mean()

TypeError: Could not convert clear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightclear-nightpartly-cloudy-nightpartly-cloudy-night to numeric

In [ ]:
len(df_resampled)

In [ ]:
df_resampled['time'] = df_resampled.index

In [ ]:
df_resampled

In [ ]:
df_resampled['use [kW]'].plot(figsize=(20,10))

In [ ]:
len(df), n_days_to_consider*24*60, len(df_resampled), len(df_resampled)*60

In [ ]:
df_resampled.columns

In [ ]:
# Measure amount of correlation with the target columns
# Specify the target column name
target_column = 'use [kW]'

# Get list of columns except the target column
other_columns = [col for col in df_resampled.columns if col != target_column]


correlations = df_resampled[other_columns].corrwith(df_resampled[target_column])
sorted_columns = correlations.sort_values(ascending=False).index

print("Sorted columns based on correlation:")
print(sorted_columns)


In [ ]:
target_series = TimeSeries.from_dataframe(df_resampled, 'time', 'use [kW]')

list_of_past_covariates = ['Furnace 2 [kW]', 'Furnace 1 [kW]', 'Living room [kW]', 'Home office [kW]', 'Fridge [kW]']
list_of_future_covariates = ['temperature', 'humidity', 'apparentTemperature', 'dewPoint']

past_covariates = TimeSeries.from_dataframe(df_resampled, 'time', list_of_past_covariates)
future_covariates = TimeSeries.from_dataframe(df_resampled, 'time', list_of_future_covariates)


In [ ]:
num_test = 24*1
target_series_train, target_series_test = target_series[:-num_test], target_series[-num_test:]

In [ ]:
n_lags = num_test*7
model = LightGBMModel(lags=n_lags, \
                      output_chunk_length=num_test, \
                      lags_past_covariates=[-n_lags]*past_covariates.n_components,\
                      lags_future_covariates=[-n_lags]*future_covariates.n_components)

In [ ]:
model.fit(series=target_series_train, \
          past_covariates=past_covariates,\
          future_covariates=future_covariates)

In [ ]:
target_series_test_pred = model.predict(num_test, \
                                        past_covariates=past_covariates,\
                                        future_covariates=future_covariates)

In [ ]:
from darts.models import NaiveSeasonal
naive_model = NaiveSeasonal(K=1)
naive_model.fit(target_series_train)
naive_forecast = naive_model.predict(num_test)

In [ ]:
#target_series_train.plot(label='training')
target_series_test.plot(label='test', lw=3)
target_series_test_pred.plot(label='LGBM', lw=3)
naive_forecast.plot(label='naive', lw=3)

In [ ]:
from darts.metrics.metrics import mape

In [ ]:
mape(target_series_test, target_series_test_pred)

In [ ]:
mape(target_series_test, naive_forecast)